In [1]:
from utility import *

In [2]:
SIZE = 256

In [19]:
# Create model and load weights

model_a = get_model(SIZE, 64)
model_b = get_model(SIZE, 64)
model_a.load_weights("aero-color_a_10k.h5")
model_b.load_weights("aero-color_b_10k.h5")

In [39]:
# Load image for bnw only
img_path = "4326.jpeg"
img = cv2.imread(img_path)
img = resize_image(img, SIZE)
X = img[:, :, 0] / 255

In [24]:
# Load image for already colored image
img_path = "color-tree.jpg"
img = cv2.imread(img_path)
img = resize_image(img, SIZE)
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l, a, b = cv2.split(lab)
X = l / 255.0

In [40]:
# Predict a and b using X
a_hats = model_a.predict(X.reshape(1, SIZE, SIZE, 1))
b_hats = model_b.predict(X.reshape(1, SIZE, SIZE, 1))

a_hats = np.log(a_hats) / 0.38
a_hats = np.exp(a_hats) / np.sum(np.exp(a_hats), axis=3, keepdims=True)
a_hats = np.matmul(a_hats, bin_val)

b_hats = np.log(b_hats) / 0.38
b_hats = np.exp(b_hats) / np.sum(np.exp(b_hats), axis=3, keepdims=True)
b_hats = np.matmul(b_hats, bin_val)

In [41]:
# Save Image
L = X.reshape((SIZE, SIZE)) * 255
a_hat = a_hats[0, :, :].astype('int')
b_hat = b_hats[0, :, :].astype('int')
lab = np.array([L, a_hat, b_hat]).transpose((1, 2, 0)).astype('uint8')
img2 = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('2. predict.jpeg', img2)

True

In [29]:
# Save the resized image for comparison
cv2.imwrite('2. original.jpeg', L)

True

In [28]:
# Save a and b for showing details
cv2.imwrite('a_our.jpeg', a)
cv2.imwrite('b_our.jpeg', b)

True